In [24]:
def read_board(file="input.txt"):
    board = []
    with open(file) as f:
        for line in f:
            board.append(list(line.strip()))
            for i in range(len(board[-1])):
                if board[-1][i] == "E":
                    end = (len(board) - 1, i)
                elif board[-1][i] == "S":
                    start = (len(board) - 1, i)
    return board, start, end

In [20]:
from heapq import *

board, start, end = read_board()
dir_dict = {
    (0, 1): [(-1, 0), (1, 0), (0, 1)],
    (1, 0): [(0, -1), (0, 1), (1, 0)],
    (0, -1): [(-1, 0), (1, 0), (0, -1)],
    (-1, 0): [(0, -1), (0, 1), (-1, 0)],
}
direction_list = [(0, 1), (1, 0), (0, -1), (-1, 0)]
dir_to_idx = {
    (0, 1): 0,
    (1, 0): 1,
    (0, -1): 2,
    (-1, 0): 3,
}
heapify(heap := [])
dist = [[[float("inf")] * 4 for _ in range(len(board[0]))] for __ in range(len(board))]

for dy, dx in [(0, 1), (1, 0), (-1, 0), (0, -1)]:
    ny, nx = start[0] + dy, start[1] + dx
    if board[ny][nx] != "#":
        idx = dir_to_idx[(dy, dx)]
        dist[ny][nx][idx] = 1 if idx == 0 else 1001
        heappush(heap, (dist[ny][nx][idx], ny, nx, (dy, dx)))

min_score = float("inf")

while heap:
    score, y, x, direction = heappop(heap)
    d_idx = dir_to_idx[direction]
    if score > dist[y][x][d_idx]:
        continue
    if board[y][x] == "E":
        min_score = min(min_score, score)
        break
    if board[y][x] != "." and board[y][x] != "S":
        continue
    for dy, dx in dir_dict[direction]:
        ny, nx = y + dy, x + dx
        if board[ny][nx] != "#":
            idx = dir_to_idx[(dy, dx)]
            new_score = score + (1 if idx == d_idx else 1001)
            if new_score < dist[ny][nx][idx]:
                dist[ny][nx][idx] = new_score
                heappush(heap, (new_score, ny, nx, (dy, dx)))
print(min_score)

92432


In [26]:
from heapq import *
from collections import Counter

board, start, end = read_board('input.txt')
dir_dict = {
    (0, 1): [(-1, 0), (1, 0), (0, 1)],
    (1, 0): [(0, -1), (0, 1), (1, 0)],
    (0, -1): [(-1, 0), (1, 0), (0, -1)],
    (-1, 0): [(0, -1), (0, 1), (-1, 0)],
}
direction_list = [(0, 1), (1, 0), (0, -1), (-1, 0)]
dir_to_idx = {
    (0, 1): 0,
    (1, 0): 1,
    (0, -1): 2,
    (-1, 0): 3,
}
heapify(heap := [])
dist = [[[float("inf")] * 4 for _ in range(len(board[0]))] for __ in range(len(board))]
pred = [[[[] for _ in range(4)] for _ in range(len(board[0]))] for __ in range(len(board))]

for dy, dx in [(0, 1), (1, 0), (-1, 0), (0, -1)]:
    ny, nx = start[0] + dy, start[1] + dx
    if board[ny][nx] != "#":
        idx = dir_to_idx[(dy, dx)]
        dist[ny][nx][idx] = 1 if idx == 0 else 1001
        pred[ny][nx][idx] = [(start[0], start[1], -1)]
        heappush(heap, (dist[ny][nx][idx], ny, nx, (dy, dx)))

min_score = float("inf")

while heap:
    score, y, x, direction = heappop(heap)
    d_idx = dir_to_idx[direction]
    if score > min_score:
        break
    if score > dist[y][x][d_idx]:
        continue
    if board[y][x] == "E":
        min_score = min(min_score, score)
        continue
    if board[y][x] != "." and board[y][x] != "S":
        continue
    for dy, dx in dir_dict[direction]:
        ny, nx = y + dy, x + dx
        if board[ny][nx] != "#":
            idx = dir_to_idx[(dy, dx)]
            new_score = score + (1 if idx == d_idx else 1001)
            if new_score < dist[ny][nx][idx]:
                dist[ny][nx][idx] = new_score
                pred[ny][nx][idx] = [(y, x, d_idx)]
                heappush(heap, (new_score, ny, nx, (dy, dx)))
            elif new_score == dist[ny][nx][idx]:
                pred[ny][nx][idx].append((y, x, d_idx))
print(min_score)

ey, ex = end


def backtrack_all_paths(y, x, d_idx):
    if not pred[y][x][d_idx]:
        return []

    paths = []
    for py, px, pd_idx in pred[y][x][d_idx]:
        if pd_idx == -1:
            paths.append([(py, px), (y, x)])
        else:
            sub_paths = backtrack_all_paths(py, px, pd_idx)
            for sp in sub_paths:
                paths.append(sp + [(y, x)])
    return paths


all_min_paths = []
for d_idx in range(4):
    if dist[ey][ex][d_idx] == min_score:
        partial = backtrack_all_paths(ey, ex, d_idx)
        all_min_paths.extend(partial)

unique_paths = list({tuple(path) for path in all_min_paths})
path_cells = set()
for path in unique_paths:
    for ry, rx in path:
        if board[ry][rx] not in ["S", "E"]:
            path_cells.add((ry, rx))

for ry, rx in path_cells:
    board[ry][rx] = "O"

print(Counter([board[y][x] for y in range(len(board)) for x in range(len(board[0]))])["O"])
print("\n".join(["".join(row) for row in board]))

92432
456
#############################################################################################################################################
#...#...........#...............#...#.....................#.............#...#.......#.#.............................#.............#.....#OOE#
#.#.#.#########.#.#####.###.###.###.#.#######.###.#######.#####.#####.#.###.#.###.#.#.#.#########.#######.#.###.#.#.#.###.#######.#.#.#.#O#.#
#.#.#.........................#...#.#...#.....#.#.......#.....#.....#.#.#.....#.#.#...#.......#...................#.#.#.#.#.......#.#.#..O#.#
#.###.#.#####.#######.#####.#.###.#.#.###.#####.#.#.#########.#######.#.#.###.#.#.#######.#.###.###.###.#.#.#.#####.#.#.#.###.###.#.#.###O###
#.#...#.#.....#.......#.....#...#...#.#...#.....#.#.........#.........#.#.......#.........#.....#.#...#.#...#.....#.#...#...#.#.....#.#OOO..#
#.#.###.#.###.#.#####.#.#.#####.###.#.#.###.###.#.#####.#.#####.#######.#############.#####.#####.###.#.#######.#.#.#######.#.#.#####.#O##